In [13]:
from matplotlib import pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

plt.style.use('Solarize_Light2')

- **Load CPILFESL data**

In [14]:
CPILFESL_df = pd.read_csv('../data/CPILFESL.csv')

CPILFESL_df['DATE'] = pd.to_datetime(CPILFESL_df['DATE'])
CPILFESL_df.set_index('DATE', inplace=True)

CPILFESL_df['^ %'] = CPILFESL_df['CPILFESL'].ffill().pct_change()

CPILFESL_df.tail()

,CPILFESL,^ %
DATE,,
2025-08-01,329.793,0.003460
2025-09-01,330.542,0.002271
2025-10-01,NaN,0.000000
2025-11-01,331.068,0.001591
2025-12-01,331.860,0.002392


- **Average change**

In [15]:
print(f"Average change is {CPILFESL_df['^ %'].mean() * 100}%")

Average change is 0.2975699021397251%


- **Overview on change**

In [16]:
positive_filter = CPILFESL_df['^ %'] > 0
negative_filter = CPILFESL_df['^ %'] < 0
zero_filter = CPILFESL_df['^ %'] == 0

# count
total_items = len(CPILFESL_df)

# average change
avg_change = [
    CPILFESL_df[positive_filter]['^ %'].mean(),
    CPILFESL_df[negative_filter]['^ %'].mean(),
    0,
    ""
]

# frequency
frequency = [
    CPILFESL_df[positive_filter]['^ %'].count(),
    CPILFESL_df[negative_filter]['^ %'].count(),
    CPILFESL_df[zero_filter]['^ %'].count()
]
frequency.append(frequency[0]/frequency[1])

# frequency pct
frequency_pct = [100 * frequency[i]/total_items for i in range(3)]
frequency_pct.append("")

# Prob adj
prob_adj = [frequency_pct[i] * avg_change[i] for i in range(3)]
prob_adj.append("")

pd.DataFrame({
    "%": [x * 100 for x in avg_change],
    "Frequency": frequency,
    "Frequency %": frequency_pct,
    "Prob Adjust % Change": prob_adj,
}, index=["Av Pos", "Av Neg", "Zero", "Ratio P/N"])

,%,Frequency,Frequency %,Prob Adjust % Change
Av Pos,0.334892,742.000000,89.613527,0.300108
Av Neg,-0.199939,12.000000,1.449275,-0.002898
Zero,0,73.000000,8.816425,0.0
Ratio P/N,,61.833333,,


- **Stats**

In [17]:
from scipy.stats import describe

# Display the results

stats = describe(CPILFESL_df['^ %'].dropna().tolist())
pd.DataFrame(
    {
        'value': [
            str(stats.nobs),
            stats.minmax[0] * 100,
            stats.minmax[1] * 100,
            stats.mean * 100,
            CPILFESL_df['^ %'].median() * 100,
            CPILFESL_df['^ %'].mode(dropna=True)[0] * 100,
            stats.variance,
            stats.skewness,
            stats.kurtosis
        ]
    },
    index=['nobs', 'Min %', 'Max %', 'Mean %', "Median %", "Mode %", 'Variance', 'Skewness', 'Kurtosis'],
)

,value
nobs,827
Min %,-0.489789
Max %,1.419355
Mean %,0.29757
Median %,0.254615
Mode %,0.0
Variance,0.000006
Skewness,1.319661
Kurtosis,3.10867


- **Data preview**

In [18]:
# Define bins
bins = [0.005 * i for i in range(0, 3)]
bins.append(stats.minmax[1])
bins.insert(0, stats.minmax[0])
bin_labels = [f"{round(bins[i] * 100, 2)}% to {round(bins[i+1] * 100, 2)}%" for i in range(len(bins) - 1)]
# bin_labels[0] = f"Less than {bins[1] * 100}%"
bin_labels[-1] = f"Greater than {bins[-2] * 100}%"


# Assign data to bins
binned = pd.cut(CPILFESL_df['^ %'], bins=bins, labels=bin_labels, include_lowest=True)

# Calculate frequency, probability, and cumulative probability
frequency = binned.value_counts().sort_index()
probability = 100 * frequency / frequency.sum()
cumulative_probability = probability.cumsum()

occurrence_frequencies = pd.DataFrame({
    'Frequency': frequency.values,
    'Probability %': probability.values,
    'Cumulative Probability %': cumulative_probability.values
}, index=bin_labels)

occurrence_frequencies

,Frequency,Probability %,Cumulative Probability %
-0.49% to 0.0%,85,10.278114,10.278114
0.0% to 0.5%,606,73.276904,83.555018
0.5% to 1.0%,117,14.147521,97.702539
Greater than 1.0%,19,2.297461,100.000000


In [19]:
fig = px.bar(occurrence_frequencies, x=occurrence_frequencies.index, y='Probability %')

fig = px.bar(
    occurrence_frequencies,
    x=occurrence_frequencies.index,
    y='Probability %',
)

fig.update_layout(
    title=dict(text='CPILFESL % change'),
    width=800,
    height=600,
    yaxis=dict(title="Probability %"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()

- CPILFESL pct change

In [20]:
fig = go.Figure()

# Reference line
fig.add_trace(go.Scatter(
    x=CPILFESL_df.index,
    y=[100 * stats.mean] * (stats.nobs + 1),
    name='Average % change',
    line=dict(
        width=2,
        color="#F2C14E",   # muted gold
        dash="dash"
    ),
    hoverinfo="skip",
    # showlegend=False,
))

# Bars
fig.add_trace(go.Bar(
    x=CPILFESL_df.index,
    y=CPILFESL_df['^ %'] * 100,
    name='CPILFESL',
    marker=dict(
        color="#4C78A8",  # professional blue
        line=dict(color="#4C78A8", width=0)
    ),
    width=0.5,
))

fig.update_layout(
    title=dict(
        text='CPILFESL pct change',
        x=0.02,
        font=dict(size=20)
    ),
    plot_bgcolor="#0E1117",
    paper_bgcolor="#0E1117",
    font=dict(color="#E6E6E6", size=14),
    width=1400,
    height=700,
    yaxis=dict(
        title="%",
        exponentformat="none",
        showgrid=False,
        zeroline=False
    ),
    xaxis=dict(
        showgrid=False,
        zeroline=False
    ),
    hovermode="x unified",
)

fig.show()

- **CPILFESL data plot**

In [21]:
fig = px.line(
    CPILFESL_df,
    x=CPILFESL_df.index,
    y="CPILFESL",
    markers=True
)

fig.update_layout(
    title=dict(text='Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average'),
    width=1000,
    height=600,
    yaxis=dict(title='CPILFESL'),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()